In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 29 kB/s 
     |████████████████████████████████| 198 kB 42.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=67f7637524ccfb9d80e38cee17eb2cafd9c6b761a807630be006cde980199190
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.master("local").appName("Practices").getOrCreate()

In [ ]:
spark

In [ ]:
df = spark.read.csv('input.csv', inferSchema = True, header = True)

In [ ]:
df.printSchema()

root
 |-- MSSV: integer (nullable = true)
 |-- NMDT: double (nullable = true)
 |-- CTRR: double (nullable = true)
 |-- HTS: double (nullable = true)
 |-- KTLT: double (nullable = true)
 |-- CTDLGT: double (nullable = true)
 |-- KTMT: double (nullable = true)
 |-- LTHDT: double (nullable = true)
 |-- MHH: double (nullable = true)
 |-- HCSDL: double (nullable = true)
 |-- HDH: double (nullable = true)
 |-- NMTTNT: integer (nullable = true)
 |-- DHMT: integer (nullable = true)
 |-- MMANM: integer (nullable = true)



In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
df.columns

['MSSV',
 'NMDT',
 'CTRR',
 'HTS',
 'KTLT',
 'CTDLGT',
 'KTMT',
 'LTHDT',
 'MHH',
 'HCSDL',
 'HDH',
 'NMTTNT',
 'DHMT',
 'MMANM']

In [ ]:
assembler = VectorAssembler(inputCols = ['NMDT','CTRR','HTS','KTLT','CTDLGT','KTMT','LTHDT','MHH','HCSDL','HDH'],
                           outputCol = 'features')

In [ ]:
output = assembler.transform(df)

In [ ]:
output.show(truncate = False)

+-------+----+----+---+----+------+----+-----+----+-----+---+------+----+-----+-------------------------------------------+
|MSSV   |NMDT|CTRR|HTS|KTLT|CTDLGT|KTMT|LTHDT|MHH |HCSDL|HDH|NMTTNT|DHMT|MMANM|features                                   |
+-------+----+----+---+----+------+----+-----+----+-----+---+------+----+-----+-------------------------------------------+
|75321  |6.5 |6.5 |5.0|6.0 |5.5   |6.5 |7.5  |7.0 |8.0  |6.5|1     |0   |1    |[6.5,6.5,5.0,6.0,5.5,6.5,7.5,7.0,8.0,6.5]  |
|81657  |7.5 |7.0 |5.5|7.0 |7.0   |7.5 |7.5  |6.0 |7.5  |7.5|1     |0   |0    |[7.5,7.0,5.5,7.0,7.0,7.5,7.5,6.0,7.5,7.5]  |
|209312 |7.0 |6.5 |5.5|7.5 |4.5   |7.0 |0.0  |7.0 |6.0  |8.0|1     |1   |0    |[7.0,6.5,5.5,7.5,4.5,7.0,0.0,7.0,6.0,8.0]  |
|583833 |6.5 |7.0 |6.0|5.5 |6.5   |7.0 |8.0  |7.0 |6.0  |7.0|1     |1   |1    |[6.5,7.0,6.0,5.5,6.5,7.0,8.0,7.0,6.0,7.0]  |
|1509861|7.5 |5.0 |5.0|9.0 |3.5   |6.5 |6.0  |7.0 |7.5  |6.5|0     |0   |0    |[7.5,5.0,5.0,9.0,3.5,6.5,6.0,7.0,7.5,6.5]  |
|2047807

In [ ]:
output.select('features','NMTTNT','DHMT','MMANM').show(truncate = False)

+-------------------------------------------+------+----+-----+
|features                                   |NMTTNT|DHMT|MMANM|
+-------------------------------------------+------+----+-----+
|[6.5,6.5,5.0,6.0,5.5,6.5,7.5,7.0,8.0,6.5]  |1     |0   |1    |
|[7.5,7.0,5.5,7.0,7.0,7.5,7.5,6.0,7.5,7.5]  |1     |0   |0    |
|[7.0,6.5,5.5,7.5,4.5,7.0,0.0,7.0,6.0,8.0]  |1     |1   |0    |
|[6.5,7.0,6.0,5.5,6.5,7.0,8.0,7.0,6.0,7.0]  |1     |1   |1    |
|[7.5,5.0,5.0,9.0,3.5,6.5,6.0,7.0,7.5,6.5]  |0     |0   |0    |
|[6.5,6.5,7.5,9.5,8.5,6.5,8.5,7.5,8.5,8.5]  |1     |1   |0    |
|[9.0,8.0,9.5,10.0,9.0,8.0,8.0,10.0,8.0,8.5]|0     |0   |0    |
|[8.5,7.0,7.0,7.0,6.0,8.0,7.0,6.5,7.5,8.0]  |0     |0   |0    |
|[7.5,8.5,8.5,10.0,8.5,8.0,8.0,8.5,0.0,9.0] |0     |0   |0    |
|[5.5,7.0,7.0,7.5,7.5,6.0,8.0,7.0,7.5,7.5]  |1     |1   |1    |
|[8.0,7.0,6.0,7.5,5.0,7.0,6.5,6.5,0.0,5.5]  |0     |0   |0    |
|[7.5,8.0,6.5,9.0,8.5,7.5,7.0,8.5,8.0,7.0]  |0     |1   |0    |
|[7.0,6.5,6.0,6.5,7.5,6.0,7.5,5.5,7.5,6.

In [ ]:
model_df = output.select('features', 'NMTTNT', 'DHMT', 'MMANM')

In [ ]:
train_df, test_df = model_df.randomSplit([0.8,0.2])

In [ ]:
train_df.count()

287

In [ ]:
test_df.count()

72

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
def DeciTree(tenmon):
    df_classifier = DecisionTreeClassifier(labelCol=tenmon).fit(train_df)
    df_predict = df_classifier.transform(test_df)
    df_predict.select(tenmon, 'prediction').show()
    df_accuracy = MulticlassClassificationEvaluator(labelCol=tenmon,
                                               metricName='accuracy').evaluate(df_predict)
    print("Accuracy: ", df_accuracy)
    df_precision = MulticlassClassificationEvaluator(labelCol='NMTTNT',
                                                metricName='weightedPrecision').evaluate(df_predict)
    print("Precision: ", df_precision)
    return df_classifier

In [ ]:
DeciTree('NMTTNT').featureImportances

+------+----------+
|NMTTNT|prediction|
+------+----------+
|     1|       1.0|
|     1|       1.0|
|     1|       1.0|
|     0|       1.0|
|     1|       1.0|
|     0|       0.0|
|     1|       1.0|
|     0|       1.0|
|     1|       1.0|
|     1|       1.0|
|     1|       1.0|
|     1|       1.0|
|     1|       1.0|
|     1|       1.0|
|     1|       1.0|
|     1|       1.0|
|     0|       0.0|
|     0|       1.0|
|     0|       1.0|
|     1|       1.0|
+------+----------+
only showing top 20 rows

Accuracy:  0.75
Precision:  0.7515873015873017


SparseVector(10, {0: 0.4082, 1: 0.14, 2: 0.043, 3: 0.0351, 4: 0.0563, 5: 0.0872, 6: 0.0139, 7: 0.0858, 9: 0.1305})

In [ ]:
DeciTree('DHMT').featureImportances

+----+----------+
|DHMT|prediction|
+----+----------+
|   1|       0.0|
|   1|       1.0|
|   1|       1.0|
|   0|       1.0|
|   1|       1.0|
|   0|       0.0|
|   0|       1.0|
|   0|       1.0|
|   1|       0.0|
|   1|       0.0|
|   1|       1.0|
|   1|       0.0|
|   1|       1.0|
|   0|       0.0|
|   1|       1.0|
|   1|       1.0|
|   0|       0.0|
|   0|       0.0|
|   0|       0.0|
|   1|       0.0|
+----+----------+
only showing top 20 rows

Accuracy:  0.7222222222222222
Precision:  0.7247474747474747


SparseVector(10, {0: 0.281, 1: 0.0563, 2: 0.1813, 3: 0.0956, 5: 0.1424, 6: 0.1706, 9: 0.0728})

In [ ]:
DeciTree('MMANM').featureImportances

+-----+----------+
|MMANM|prediction|
+-----+----------+
|    1|       1.0|
|    0|       1.0|
|    0|       1.0|
|    0|       1.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 20 rows

Accuracy:  0.7638888888888888
Precision:  0.5669154228855722


SparseVector(10, {0: 0.2868, 1: 0.04, 2: 0.0135, 4: 0.3274, 5: 0.2102, 7: 0.1222})

In [ ]:
dt_fit_NMTTNT = DeciTree('NMTTNT')

In [ ]:
df_fit_DHMT = DeciTree('DHMT')
df_fit_MMANM = DeciTree('MMANM')

In [ ]:
# The following code lines only help visualizing DTs.

In [ ]:
dt_fit_NMTTNT.toDebugString

'DecisionTreeClassificationModel: uid=DecisionTreeClassifier_da173ffc8de9, depth=5, numNodes=31, numClasses=2, numFeatures=10\n  If (feature 0 <= 7.25)\n   If (feature 9 <= 4.5)\n    Predict: 0.0\n   Else (feature 9 > 4.5)\n    If (feature 7 <= 8.25)\n     If (feature 2 <= 3.75)\n      If (feature 0 <= 6.25)\n       Predict: 0.0\n      Else (feature 0 > 6.25)\n       Predict: 1.0\n     Else (feature 2 > 3.75)\n      Predict: 1.0\n    Else (feature 7 > 8.25)\n     If (feature 6 <= 9.25)\n      Predict: 0.0\n     Else (feature 6 > 9.25)\n      Predict: 1.0\n  Else (feature 0 > 7.25)\n   If (feature 5 <= 7.25)\n    If (feature 1 <= 6.75)\n     If (feature 2 <= 7.75)\n      Predict: 0.0\n     Else (feature 2 > 7.75)\n      Predict: 1.0\n    Else (feature 1 > 6.75)\n     If (feature 9 <= 7.75)\n      If (feature 3 <= 6.75)\n       Predict: 1.0\n      Else (feature 3 > 6.75)\n       Predict: 0.0\n     Else (feature 9 > 7.75)\n      Predict: 1.0\n   Else (feature 5 > 7.25)\n    If (feature 0 

In [ ]:
def parse_debug_string_lines(lines):
    
    block = []
    while lines:


        if lines[0].startswith('If'):
            bl = ' '.join(lines.pop(0).split()[1:]).replace('(', '').replace(')', '')
            block.append({'name': bl, 'children': parse_debug_string_lines(lines)})


            if lines[0].startswith('Else'):
                be = ' '.join(lines.pop(0).split()[1:]).replace('(', '').replace(')', '')
                block.append({'name': be, 'children': parse_debug_string_lines(lines)})
        elif not lines[0].startswith(('If', 'Else')):
            block2 = lines.pop(0)
            block.append({'name': block2})
        else:
            break
    
    return block


def debug_str_to_json(debug_string):
    data = []
    for line in debug_string.splitlines():
        if line.strip():
            line = line.strip()
            data.append(line)
        else:
            break
        if not line: break

    json = {'name': 'Root', 'children': parse_debug_string_lines(data[1:])}
    

    return json

In [ ]:
import json

dict_tree_json = debug_str_to_json(dt_fit_NMTTNT.toDebugString)

print(json.dumps(dict_tree_json,indent = 1 ))

{
 "name": "Root",
 "children": [
  {
   "name": "feature 0 <= 7.25",
   "children": [
    {
     "name": "feature 9 <= 4.5",
     "children": [
      {
       "name": "Predict: 0.0"
      }
     ]
    },
    {
     "name": "feature 9 > 4.5",
     "children": [
      {
       "name": "feature 7 <= 8.25",
       "children": [
        {
         "name": "feature 2 <= 3.75",
         "children": [
          {
           "name": "feature 0 <= 6.25",
           "children": [
            {
             "name": "Predict: 0.0"
            }
           ]
          },
          {
           "name": "feature 0 > 6.25",
           "children": [
            {
             "name": "Predict: 1.0"
            }
           ]
          }
         ]
        },
        {
         "name": "feature 2 > 3.75",
         "children": [
          {
           "name": "Predict: 1.0"
          }
         ]
        }
       ]
      },
      {
       "name": "feature 7 > 8.25",
       "children": [
        {
       

In [ ]:
f_type_to_flist_dict = output.schema['features'].metadata["ml_attr"]["attrs"]

f_index_to_name_dict = {}

for f_type, f_list in f_type_to_flist_dict.items():
  for f in f_list:
    f_index = f['idx']
    f_name = f['name']
    f_index_to_name_dict[f_index] = f_name




print(f_index_to_name_dict)

{0: 'NMDT', 1: 'CTRR', 2: 'HTS', 3: 'KTLT', 4: 'CTDLGT', 5: 'KTMT', 6: 'LTHDT', 7: 'MHH', 8: 'HCSDL', 9: 'HDH'}


In [ ]:
dict_tree_json = debug_str_to_json(df_fit_MMANM.toDebugString)

print(json.dumps(dict_tree_json,indent = 1 ))

{
 "name": "Root",
 "children": [
  {
   "name": "feature 0 <= 7.25",
   "children": [
    {
     "name": "feature 4 <= 5.25",
     "children": [
      {
       "name": "feature 4 <= 3.75",
       "children": [
        {
         "name": "feature 4 <= 1.25",
         "children": [
          {
           "name": "Predict: 0.0"
          }
         ]
        },
        {
         "name": "feature 4 > 1.25",
         "children": [
          {
           "name": "feature 5 <= 5.75",
           "children": [
            {
             "name": "Predict: 1.0"
            }
           ]
          },
          {
           "name": "feature 5 > 5.75",
           "children": [
            {
             "name": "Predict: 0.0"
            }
           ]
          }
         ]
        }
       ]
      },
      {
       "name": "feature 4 > 3.75",
       "children": [
        {
         "name": "Predict: 0.0"
        }
       ]
      }
     ]
    },
    {
     "name": "feature 4 > 5.25",
     "child